In [4]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras.layers import Input, Dense, Reshape, Flatten
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
from keras.models import Sequential, Model
from keras.layers import Conv2DTranspose
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator


In [5]:
# Define your dataset directory
dataset_dir = 'C:\\Users\\Rishikeshs\\Downloads\\Cat'

# Define parameters
img_width, img_height, channels = 28, 28, 3  # Adjust these dimensions to match your images
latent_dim = 100  # Size of the random noise vector
batch_size = 128
epochs = 200
save_interval = 100  # Save generated images every 'save_interval' epochs

# Create a data generator for loading and augmenting images
datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,  # Normalize pixel values to [0, 1]
    horizontal_flip=True  # Data augmentation: flip images horizontally
)

# Load and preprocess images from your dataset using the data generator
data_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None  # We don't need class labels for a GAN
)

# Build the generator network
generator = Sequential()
generator.add(Dense(128 * 16 * 16, input_dim=latent_dim))
generator.add(LeakyReLU(0.2))
generator.add(Reshape((16, 16, 128)))
generator.add(BatchNormalization())
generator.add(Dense(128 * 32 * 32))
generator.add(LeakyReLU(0.2))
generator.add(Reshape((32, 32, 128)))
generator.add(BatchNormalization())
generator.add(Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same'))
generator.add(LeakyReLU(0.2))
generator.add(Conv2DTranspose(channels, (5, 5), strides=(2, 2), padding='same', activation='sigmoid'))

# Build the discriminator network
discriminator = Sequential()
discriminator.add(Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=(img_width, img_height, channels)))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Flatten())
discriminator.add(Dense(1, activation='sigmoid'))

# Compile the discriminator
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5), metrics=['accuracy'])

# Build the GAN model by combining the generator and discriminator
discriminator.trainable = False  # Freeze the discriminator during GAN training
gan_input = Input(shape=(latent_dim,))
x = generator(gan_input)
gan_output = discriminator(x)
gan = Model(gan_input, gan_output)
gan.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5))

# Training loop
for epoch in range(epochs):
    for _ in range(data_generator.samples // batch_size):
        # Train discriminator on real images
        real_images = next(data_generator)
        real_labels = np.ones((batch_size, 1))
        d_loss_real = discriminator.train_on_batch(real_images, real_labels)

        # Train discriminator on generated (fake) images
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        fake_images = generator.predict(noise)
        fake_labels = np.zeros((batch_size, 1))
        d_loss_fake = discriminator.train_on_batch(fake_images, fake_labels)

        # Train generator to fool the discriminator
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        valid_labels = np.ones((batch_size, 1))
        g_loss = gan.train_on_batch(noise, valid_labels)

    # Print progress and save generated images
    if epoch % save_interval == 0:
        print(f"Epoch {epoch}, D Loss Real: {d_loss_real[0]}, D Loss Fake: {d_loss_fake[0]}, G Loss: {g_loss}")
        save_generated_images(epoch)

# Function to save generated images
def save_generated_images(epoch, examples=10, dim=(1, 10), figsize=(10, 1)):
    noise = np.random.normal(0, 1, (examples, latent_dim))
    generated_images = generator.predict(noise)
    generated_images = 0.5 * generated_images + 0.5  # Rescale to [0, 1]

    plt.figure(figsize=figsize)
    for i in range(examples):
        plt.subplot(dim[0], dim[1], i + 1)
        plt.imshow(generated_images[i], interpolation='nearest')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig(f'gan_generated_image_epoch_{epoch}.png')
    plt.close()

Found 0 images belonging to 0 classes.


ValueError: Exception encountered when calling layer "reshape_5" (type Reshape).

total size of new array must be unchanged, input_shape = [16, 16, 131072], output_shape = [32, 32, 128]

Call arguments received by layer "reshape_5" (type Reshape):
  • inputs=tf.Tensor(shape=(None, 16, 16, 131072), dtype=float32)